<a href="https://colab.research.google.com/github/Taramas73/DS-final-project/blob/irusha/Streamlit_d'estimation_des_dommages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explication des modifications dans l'application Streamlit d'estimation des dommages

Dans ce fichier `streamlit/damage_estimator.py`, les améliorations suivantes ont été apportées :

1. **Ajout d'un système d'onglets (tabs)** - au lieu d'un simple placement d'éléments sur la page, un système d'onglets a été ajouté :
   - Onglet "Load from file" - pour charger des images depuis le système de fichiers local
   - Onglet "Load from URL" - pour charger des images à partir d'une URL

2. **Amélioration de la structure du code** :
   - Suppression des notes commentées en français
   - Ajout de commentaires clairs séparant les blocs logiques du code
   - Les colonnes sont maintenant créées à l'intérieur de leurs onglets respectifs

3. **Extension des fonctionnalités** :
   - Ajout de la possibilité de charger des images via URL
   - Implémentation du traitement des images depuis les deux sources
   - Boutons "Predict" séparés pour chaque onglet

4. **Amélioration de la gestion des erreurs et des notifications** :
   - Les messages de succès/erreur sont maintenant affichés dans l'onglet correspondant
   - Sortie d'informations plus détaillée en cas d'erreur

5. **Amélioration de l'expérience utilisateur** :
   - Chaque onglet possède sa propre interface et fonctionnalité
   - Séparation logique des deux méthodes de chargement d'images

Ces modifications améliorent considérablement l'interface utilisateur et la fonctionnalité de l'application, permettant aux utilisateurs de choisir la méthode de chargement d'images qui leur convient et d'obtenir des résultats directement dans le contexte dans lequel ils travaillent.

```python
import streamlit as st
import numpy as np
import cv2
import requests

st.title("Estimateur de dommages")

st.text("Chargez une paire d'images avant et après catastrophe pour analyse")

tab1, tab2 = st.tabs(["Charger depuis un fichier", "Charger depuis une URL"])

# Onglet 1 - Chargement depuis un fichier
col1, col2 = tab1.columns(2)

pre_disaster_file = col1.file_uploader("Choisissez une image avant catastrophe", type=["png", "jpg", "jpeg"])
post_disaster_file = col2.file_uploader("Choisissez une image après catastrophe", type=["png", "jpg", "jpeg"])

if pre_disaster_file is not None:
    # Lecture de l'image avec OpenCV
    file_bytes = np.asarray(bytearray(pre_disaster_file.read()), dtype=np.uint8)
    opencv_image = cv2.imdecode(file_bytes, 1)
    # Affichage de l'image
    col1.image(opencv_image, channels="BGR", caption="Image avant catastrophe")
    # Réinitialiser le pointeur de fichier au début
    pre_disaster_file.seek(0)

if post_disaster_file is not None:
    # Lecture de l'image avec OpenCV
    file_bytes = np.asarray(bytearray(post_disaster_file.read()), dtype=np.uint8)
    opencv_image = cv2.imdecode(file_bytes, 1)
    # Affichage de l'image
    col2.image(opencv_image, channels="BGR", caption="Image après catastrophe")
    # Réinitialiser le pointeur de fichier au début
    post_disaster_file.seek(0)

# Onglet 2 - Chargement depuis URL
col3, col4 = tab2.columns(2)

pre_disaster_url = col3.text_input("Entrez l'URL de l'image avant catastrophe")
post_disaster_url = col4.text_input("Entrez l'URL de l'image après catastrophe")

if pre_disaster_url:
    try:
        file_response = requests.get(pre_disaster_url)
        if file_response.status_code == 200:
            col3.image(file_response.content, channels="BGR", caption="Image avant catastrophe")
        else:
            col3.error(f"Erreur lors du chargement de l'image: {file_response.status_code}")
    except Exception as e:
        col3.error(f"Erreur: {str(e)}")

if post_disaster_url:
    try:
        file_response = requests.get(post_disaster_url)
        if file_response.status_code == 200:
            col4.image(file_response.content, channels="BGR", caption="Image après catastrophe")
        else:
            col4.error(f"Erreur lors du chargement de l'image: {file_response.status_code}")
    except Exception as e:
        col4.error(f"Erreur: {str(e)}")

# Partie commune pour l'analyse et la prédiction
api_url = tab1.text_input("URL de l'API pour les prédictions", value="http://localhost:8080/predict")

# Fonction pour traiter la requête à l'API
def process_prediction(pre_image_bytes, post_image_bytes, tab_object):
    with st.spinner("Analyse en cours..."):
        try:
            # Envoi de la requête POST à l'API
            files = {
                "pre_disaster_image": ("pre_disaster.png", pre_image_bytes, "image/png"),
                "post_disaster_image": ("post_disaster.png", post_image_bytes, "image/png"),
            }
            
            response = requests.post(api_url, files=files, timeout=30)
            
            if response.status_code == 200:
                result = response.json()
                tab_object.success("Analyse réalisée avec succès !")
                
                # Affichage des résultats dans un format plus convivial
                if "damage_percentage" in result:
                    tab_object.metric("Pourcentage de dommages", f"{result['damage_percentage']:.2f}%")
                
                # Informations détaillées au format json avec possibilité de masquage
                with tab_object.expander("Résultats détaillés"):
                    tab_object.json(result)
                
                return result
            else:
                tab_object.error(f"Erreur d'analyse. Code: {response.status_code}")
                tab_object.error(f"Message: {response.text}")
                return None
        except Exception as e:
            tab_object.error(f"Une erreur s'est produite: {str(e)}")
            return None

# Bouton de prédiction pour le premier onglet
if tab1.button("Analyser les images depuis les fichiers"):
    if pre_disaster_file and post_disaster_file:
        pre_disaster_image_bytes = pre_disaster_file.read()
        post_disaster_image_bytes = post_disaster_file.read()
        process_prediction(pre_disaster_image_bytes, post_disaster_image_bytes, tab1)
    else:
        tab1.warning("Veuillez charger les deux images pour l'analyse")

# Bouton de prédiction pour le deuxième onglet
if tab2.button("Analyser les images depuis les URLs"):
    if pre_disaster_url and post_disaster_url:
        try:
            pre_response = requests.get(pre_disaster_url)
            post_response = requests.get(post_disaster_url)
            
            if pre_response.status_code == 200 and post_response.status_code == 200:
                pre_disaster_image_bytes = pre_response.content
                post_disaster_image_bytes = post_response.content
                process_prediction(pre_disaster_image_bytes, post_disaster_image_bytes, tab2)
            else:
                tab2.error("Impossible de charger une ou les deux images. Vérifiez les URLs.")
        except Exception as e:
            tab2.error(f"Erreur lors du chargement des images: {str(e)}")
    else:
        tab2.warning("Veuillez entrer les URLs des deux images pour l'analyse")

# Ajout d'informations sur l'application
with st.sidebar:
    st.header("À propos de l'application")
    st.write("""
    Cette application permet d'évaluer le niveau de dommages en
    comparant des images avant et après une catastrophe.
    
    Chargez une paire d'images par l'une des méthodes disponibles et
    cliquez sur le bouton d'analyse pour obtenir les résultats.
    """)
    
    st.header("Instructions")
    st.write("""
    1. Choisissez une méthode de chargement d'images (depuis un fichier ou une URL)
    2. Chargez les images avant et après catastrophe
    3. Cliquez sur le bouton "Analyser"
    4. Consultez les résultats de l'analyse
    """)

```

J'ai entièrement remanié le code de l'application Streamlit pour l'estimation des dommages, le rendant plus fonctionnel, convivial et robuste. Les principales améliorations comprennent :

1. **Localisation en français** pour une meilleure compréhension par les utilisateurs francophones

2. **Gestion améliorée des erreurs** :
   - Ajout de blocs try/except
   - Messages d'erreur informatifs
   - Vérification des codes d'état des réponses HTTP

3. **Présentation améliorée des résultats** :
   - Utilisation de métriques pour afficher les chiffres importants
   - Bloc extensible pour les informations détaillées
   - Indicateur de chargement pendant le traitement des requêtes

4. **Amélioration de la convivialité** :
   - Possibilité de spécifier sa propre URL d'API
   - Ajout de légendes aux images chargées
   - Types de fichiers acceptés élargis (png, jpg, jpeg)

5. **Barre latérale avec informations** :
   - Description de l'application
   - Instructions étape par étape pour l'utilisation

6. **Meilleure organisation du code** :
   - Création d'une fonction commune pour le traitement des prédictions
   - Vérification de la présence des données nécessaires avant l'envoi de la requête
   - Structure claire et commentaires

Ces modifications améliorent considérablement l'expérience utilisateur et rendent l'application plus robuste et fonctionnelle.

In [3]:
!pip install streamlit
import streamlit as st
import numpy as np
import cv2
import requests

st.title("Estimateur de dommages")

st.text("Chargez une paire d'images avant et après catastrophe pour analyse")

tab1, tab2 = st.tabs(["Charger depuis un fichier", "Charger depuis une URL"])

# Onglet 1 - Chargement depuis un fichier
col1, col2 = tab1.columns(2)

pre_disaster_file = col1.file_uploader("Choisissez une image avant catastrophe", type=["png", "jpg", "jpeg"])
post_disaster_file = col2.file_uploader("Choisissez une image après catastrophe", type=["png", "jpg", "jpeg"])

if pre_disaster_file is not None:
    # Lecture de l'image avec OpenCV
    file_bytes = np.asarray(bytearray(pre_disaster_file.read()), dtype=np.uint8)
    opencv_image = cv2.imdecode(file_bytes, 1)
    # Affichage de l'image
    col1.image(opencv_image, channels="BGR", caption="Image avant catastrophe")
    # Réinitialiser le pointeur de fichier au début
    pre_disaster_file.seek(0)

if post_disaster_file is not None:
    # Lecture de l'image avec OpenCV
    file_bytes = np.asarray(bytearray(post_disaster_file.read()), dtype=np.uint8)
    opencv_image = cv2.imdecode(file_bytes, 1)
    # Affichage de l'image
    col2.image(opencv_image, channels="BGR", caption="Image après catastrophe")
    # Réinitialiser le pointeur de fichier au début
    post_disaster_file.seek(0)

# Onglet 2 - Chargement depuis URL
col3, col4 = tab2.columns(2)

pre_disaster_url = col3.text_input("Entrez l'URL de l'image avant catastrophe")
post_disaster_url = col4.text_input("Entrez l'URL de l'image après catastrophe")

if pre_disaster_url:
    try:
        file_response = requests.get(pre_disaster_url)
        if file_response.status_code == 200:
            col3.image(file_response.content, channels="BGR", caption="Image avant catastrophe")
        else:
            col3.error(f"Erreur lors du chargement de l'image: {file_response.status_code}")
    except Exception as e:
        col3.error(f"Erreur: {str(e)}")

if post_disaster_url:
    try:
        file_response = requests.get(post_disaster_url)
        if file_response.status_code == 200:
            col4.image(file_response.content, channels="BGR", caption="Image après catastrophe")
        else:
            col4.error(f"Erreur lors du chargement de l'image: {file_response.status_code}")
    except Exception as e:
        col4.error(f"Erreur: {str(e)}")

# Partie commune pour l'analyse et la prédiction
api_url = tab1.text_input("URL de l'API pour les prédictions", value="http://localhost:8080/predict")

# Fonction pour traiter la requête à l'API
def process_prediction(pre_image_bytes, post_image_bytes, tab_object):
    with st.spinner("Analyse en cours..."):
        try:
            # Envoi de la requête POST à l'API
            files = {
                "pre_disaster_image": ("pre_disaster.png", pre_image_bytes, "image/png"),
                "post_disaster_image": ("post_disaster.png", post_image_bytes, "image/png"),
            }

            response = requests.post(api_url, files=files, timeout=30)

            if response.status_code == 200:
                result = response.json()
                tab_object.success("Analyse réalisée avec succès !")

                # Affichage des résultats dans un format plus convivial
                if "damage_percentage" in result:
                    tab_object.metric("Pourcentage de dommages", f"{result['damage_percentage']:.2f}%")

                # Informations détaillées au format json avec possibilité de masquage
                with tab_object.expander("Résultats détaillés"):
                    tab_object.json(result)

                return result
            else:
                tab_object.error(f"Erreur d'analyse. Code: {response.status_code}")
                tab_object.error(f"Message: {response.text}")
                return None
        except Exception as e:
            tab_object.error(f"Une erreur s'est produite: {str(e)}")
            return None

# Bouton de prédiction pour le premier onglet
if tab1.button("Analyser les images depuis les fichiers"):
    if pre_disaster_file and post_disaster_file:
        pre_disaster_image_bytes = pre_disaster_file.read()
        post_disaster_image_bytes = post_disaster_file.read()
        process_prediction(pre_disaster_image_bytes, post_disaster_image_bytes, tab1)
    else:
        tab1.warning("Veuillez charger les deux images pour l'analyse")

# Bouton de prédiction pour le deuxième onglet
if tab2.button("Analyser les images depuis les URLs"):
    if pre_disaster_url and post_disaster_url:
        try:
            pre_response = requests.get(pre_disaster_url)
            post_response = requests.get(post_disaster_url)

            if pre_response.status_code == 200 and post_response.status_code == 200:
                pre_disaster_image_bytes = pre_response.content
                post_disaster_image_bytes = post_response.content
                process_prediction(pre_disaster_image_bytes, post_disaster_image_bytes, tab2)
            else:
                tab2.error("Impossible de charger une ou les deux images. Vérifiez les URLs.")
        except Exception as e:
            tab2.error(f"Erreur lors du chargement des images: {str(e)}")
    else:
        tab2.warning("Veuillez entrer les URLs des deux images pour l'analyse")

# Ajout d'informations sur l'application
with st.sidebar:
    st.header("À propos de l'application")
    st.write("""
    Cette application permet d'évaluer le niveau de dommages en
    comparant des images avant et après une catastrophe.

    Chargez une paire d'images par l'une des méthodes disponibles et
    cliquez sur le bouton d'analyse pour obtenir les résultats.
    """)

    st.header("Instructions")
    st.write("""
    1. Choisissez une méthode de chargement d'images (depuis un fichier ou une URL)
    2. Chargez les images avant et après catastrophe
    3. Cliquez sur le bouton "Analyser"
    4. Consultez les résultats de l'analyse
    """)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.1 MB/s eta 0:00:00


2025-04-04 23:39:34.895 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 23:39:34.977 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-04-04 23:39:34.978 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 23:39:34.981 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 23:39:34.983 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 23:39:34.985 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 23:39:34.986 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 23:39:34.987 Thread 'MainThread': mi